In [1]:

import os
import warnings
import pandas as pd
from crewai import Agent, Task, Crew
from crewai.tools import BaseTool
import agentops
import yaml
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool
from sqlalchemy import create_engine

warnings.filterwarnings('ignore')
from dotenv import load_dotenv
# Load environment variables
load_dotenv()

AGENTOPS_API_KEY = os.getenv('AGENTOPS_API_KEY')

agentops.init(AGENTOPS_API_KEY)

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=39877227-f9cb-4013-83b6-9833e34ee4ce


In [2]:
# Path to your SQLite database file
database_file_path = "test.db"

# Create an engine to connect to the SQLite database
# SQLite only requires the path to the database file
engine = create_engine(f'sqlite:///{database_file_path}')
file_url = "./data/netflix_titles.csv"
df = pd.read_csv(file_url).fillna(value = 0)
df.to_sql(
    'netflix_titles',
    con=engine,
    if_exists='replace',
    index=False
)

8807

In [3]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///test.db")
print(db.dialect)
print(db.get_usable_table_names())
print(db.get_table_info(['netflix_titles']))
# db.run("SELECT * FROM netflix_titles LIMIT 10;")

sqlite
['netflix_titles']

CREATE TABLE netflix_titles (
	show_id TEXT, 
	type TEXT, 
	title TEXT, 
	director TEXT, 
	"cast" TEXT, 
	country TEXT, 
	date_added TEXT, 
	release_year BIGINT, 
	rating TEXT, 
	duration TEXT, 
	listed_in TEXT, 
	description TEXT
)

/*
3 rows from netflix_titles table:
show_id	type	title	director	cast	country	date_added	release_year	rating	duration	listed_in	description
s1	Movie	Dick Johnson Is Dead	Kirsten Johnson	0	United States	September 25, 2021	2020	PG-13	90 min	Documentaries	As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and
s2	TV Show	Blood & Water	0	Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Gree	South Africa	September 24, 2021	2021	TV-MA	2 Seasons	International TV Shows, TV Dramas, TV Mysteries	After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimmin
s3	TV Show	Ganglands	Julien Leclercq	Sami Bouajila, Tra

In [4]:
def execute_query(query):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=db)
    return {"result": execute_query_tool.invoke(query)}

execute_query("describe netflix_titles;")

{'result': 'Error: (sqlite3.OperationalError) near "describe": syntax error\n[SQL: describe netflix_titles;]\n(Background on this error at: https://sqlalche.me/e/20/e3q8)'}

### Create some tools

In [5]:
class DatabaseQuery(BaseTool):
    name: str = "Database Query"
    description: str = "Returns the result of SQL query execution"

    def _run(self, sql_query: str) -> str:
        # Implementation goes here
        return execute_query(sql_query)
      
database_query_tool = DatabaseQuery()

In [6]:
class TableStructure(BaseTool):
    name: str = "Table structure"
    description: str = "Returns the list of columns and their types"

    def _run(self, table: str) -> str:
        return db.get_table_info([table])

class TableExamples(BaseTool):
    name: str = "Table examples"
    description: str = "Returns the first N rows from the table"

    def _run(self, table: str, n: int = 30) -> str:
        query = f"select * from {table} limit {n}"
        return execute_query(query)

table_structure_tool = TableStructure()
table_examples_tool = TableExamples()

In [7]:
print(table_structure_tool.run("netflix_titles"))
print(table_examples_tool.run("netflix_titles", 10))


Using Tool: Table structure

CREATE TABLE netflix_titles (
	show_id TEXT, 
	type TEXT, 
	title TEXT, 
	director TEXT, 
	"cast" TEXT, 
	country TEXT, 
	date_added TEXT, 
	release_year BIGINT, 
	rating TEXT, 
	duration TEXT, 
	listed_in TEXT, 
	description TEXT
)

/*
3 rows from netflix_titles table:
show_id	type	title	director	cast	country	date_added	release_year	rating	duration	listed_in	description
s1	Movie	Dick Johnson Is Dead	Kirsten Johnson	0	United States	September 25, 2021	2020	PG-13	90 min	Documentaries	As her father nears the end of his life, filmmaker Kirsten Johnson stages his death in inventive and
s2	TV Show	Blood & Water	0	Ama Qamata, Khosi Ngema, Gail Mabalane, Thabang Molaba, Dillon Windvogel, Natasha Thahane, Arno Gree	South Africa	September 24, 2021	2021	TV-MA	2 Seasons	International TV Shows, TV Dramas, TV Mysteries	After crossing paths at a party, a Cape Town teen sets out to prove whether a private-school swimmin
s3	TV Show	Ganglands	Julien Leclercq	Sami Bouajila, T

## Load Tasks and Agent files

In [8]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Create Agents and Tasks

In [9]:
database_specialist_agent = Agent(
    config=agents_config['database_specialist_agent'],
)

tech_writer_agent = Agent(
    config=agents_config['tech_writer_agent'],
)

qa_specialist_agent = Agent(
    config=agents_config['qa_specialist_agent'],
)

table_description_task = Task(
    config=tasks_config['table_description_task'],
    tools = [database_query_tool, table_structure_tool, table_examples_tool],
    agent = database_specialist_agent
)

table_documentation_task = Task(
    config=tasks_config['table_documentation_task'],
    tools = [],
    output_file="table_documentation.md",
    agent = tech_writer_agent
)

qa_review_task = Task(
    config=tasks_config['qa_review_task'],
    tools = [],
    context = [table_description_task, table_documentation_task],
    output_file="checked_table_documentation.md",
    agent = qa_specialist_agent
)


In [10]:
full_crew = Crew(
    agents=[database_specialist_agent, tech_writer_agent, qa_specialist_agent],
    tasks=[table_description_task,  table_documentation_task, qa_review_task],
    verbose=True,
    memory = True
)

In [11]:
#%%time

full_result = full_crew.kickoff({'table': 'netflix_titles'})

# Agent: database specialist
## Task: Provide the comprehensive overview for the data in table netflix_titles, so that it's easy to understand the structure of the data. This task is crucial to put together the documentation for our database



# Agent: database specialist
## Thought: I will start by gathering the structure of the `netflix_titles` table to understand what columns it contains and their types.
## Using tool: Table structure
## Tool Input: 
"{\"table\": \"netflix_titles\"}"
## Tool Output: 

CREATE TABLE netflix_titles (
	show_id TEXT, 
	type TEXT, 
	title TEXT, 
	director TEXT, 
	"cast" TEXT, 
	country TEXT, 
	date_added TEXT, 
	release_year BIGINT, 
	rating TEXT, 
	duration TEXT, 
	listed_in TEXT, 
	description TEXT
)

/*
3 rows from netflix_titles table:
show_id	type	title	director	cast	country	date_added	release_year	rating	duration	listed_in	description
s1	Movie	Dick Johnson Is Dead	Kirsten Johnson	0	United States	September 25, 2021	2020	PG-13	90 min	Documentaries	As

🖇 AgentOps: Session Stats - Duration: 3m 4.8s | Cost: $0.008235 | LLMs: 8 | Tools: 2 | Actions: 0 | Errors: 0
🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=39877227-f9cb-4013-83b6-9833e34ee4ce


In [ ]:
from IPython.display import Markdown
Markdown(full_result.replace("```", ""))

In [ ]:
#%%time
full_sessions_result = full_crew.kickoff({'table': 'ecommerce_db.sessions'})

In [ ]:
from IPython.display import Markdown
Markdown(full_sessions_result.replace("```", ""))